In [35]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [36]:
import imp
imp.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [26]:
# open state_fips.csv
state_fips = pd.read_csv('state_fips.csv')
state_fips.head()

,state,code,abbreviation
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA
5,Colorado,8,CO
6,Connecticut,9,CT
7,Delaware,10,DE
8,District of Columbia,11,DC
9,Florida,12,FL


In [33]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)

    # split folder into list by underscore
    folder = folder.split('_')

    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder[-1] == 'tract' else '20' + folder[-1][-2:]
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [53]:
# for alaska lower
# read in alaska_lower_2022.shp
districts = gpd.read_file(r'raw_geo_data\tl_2023_02_sldl\tl_2023_02_sldl.shp')
# only keep SLDLST and geometry
districts = districts[['SLDLST', 'geometry']]
# rename SLDLST to district and convert to int
districts = districts.rename(columns={'SLDLST': 'district'})
districts['district'] = districts['district'].astype(int)
# save to geo_polygons folder as alaska_lower_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_lower_districts_2022.csv', index=False)